In [1]:
# Library imports
import os
import re
import getpass
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.base import RunnableSequence

In [2]:
# !pip install -q langchain langchain-community langchain-groq

In [3]:
# API key set up

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")



In [4]:
# Text cleaning and transformation

def clean_transform_text(text):
  #Remove special characters and normalize whitespace
  input_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
  cleaned_text = re.sub(r'\s+', ' ', input_text).strip()
  return cleaned_text

In [5]:
# Sentiment analysis (core function)

def sentiment_analysis(text):
  # Model init
  chat_model = ChatGroq(model_name="gemma2-9b-it")
  # Prompt set up
  prompt1 = f"Analyze the sentiment of the following text and classify it as positive, negative, or neutral; without explanation: \n{text}"
  prompt2 = f"Give the certainty score of the sentiment analysis of the text without explanation. Be it from 0 to 0.59 bad, between 0.6 and 0.79 neutral, and above 0.8 good: \n{text}"
  # Model response
  sentiment = chat_model.invoke(prompt1).content.strip().lower()
  score = chat_model.invoke(prompt2).content.strip().lower()
  
  return {"text": text, "sentiment": sentiment, "model_score": score}


In [6]:
# Pipeline build-up

runnable_1 = RunnableLambda(lambda texts: [clean_transform_text(text) for text in texts])
runnable_2 = RunnableLambda(lambda clean_transform_texts: [sentiment_analysis(text) for text in clean_transform_texts])

In [7]:
# Texts to analyze

reviews = [
    "The product quality is excellent and the service was awesome!",
    "Very poor experience, the product was bad and delivery was horrible.",
    "Good value for money, but could be better.",
    "I had a bad experience, but the customer support was good.",
    "This is the best product I've ever bought!",
    "It was a useful product, hope it gets better."
]

In [8]:
# Pipeline running

custom_sequence = runnable_1 | runnable_2
review_analysis_pipeline = RunnableSequence(custom_sequence)

In [9]:
result = review_analysis_pipeline.invoke(reviews)
result

[{'text': 'The product quality is excellent and the service was awesome',
  'sentiment': 'positive',
  'model_score': '0.95'},
 {'text': 'Very poor experience the product was bad and delivery was horrible',
  'sentiment': 'negative',
  'model_score': '0.0'},
 {'text': 'Good value for money but could be better',
  'sentiment': 'neutral',
  'model_score': '0.65'},
 {'text': 'I had a bad experience but the customer support was good',
  'sentiment': 'neutral',
  'model_score': '0.5'},
 {'text': 'This is the best product Ive ever bought',
  'sentiment': 'positive',
  'model_score': '0.99'},
 {'text': 'It was a useful product hope it gets better',
  'sentiment': 'neutral',
  'model_score': '0.68'}]

In [10]:
df = pd.DataFrame.from_dict(result)
df

,text,sentiment,model_score
0,The product quality is excellent and the servi...,positive,0.95
1,Very poor experience the product was bad and d...,negative,0.0
2,Good value for money but could be better,neutral,0.65
3,I had a bad experience but the customer suppor...,neutral,0.5
4,This is the best product Ive ever bought,positive,0.99
5,It was a useful product hope it gets better,neutral,0.68
